In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import requests
import time
from datetime import datetime
from google.cloud.logging import DESCENDING, Client
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3
plt.rcParams['axes.titlesize'] = 18

# SCRIPT

In [56]:
# read getPayloads from file.
gp = pd.read_csv('gpFull.txt', delimiter=',')
gp['inserted_at'] = pd.to_datetime(gp['inserted_at'], format="%Y-%m-%d %H:%M:%S.%f")
gp.head()

,slot,ip,ua,req_time,block_hash,proposer_pubkey,inserted_at,slot_boundary
0,6200251,13.51.48.23,mev-boost/v1.5.0 Go-http-client/1.1,2023-04-11 15:30:35.808494+00:00,0xa9919dcc29eabc8e0c967108e306dfc300d2425c7912...,0x8a13257bd79486d55e39b5edba59be7db1539789b71b...,2023-04-11 15:30:37.358157,2023-04-11 15:30:35.000000
1,6200260,34.249.7.41,mev-boost/v1.5.0 Go-http-client/1.1,2023-04-11 15:32:23.554700+00:00,0x15116fbb7f527090d7f5776926c945b8123224ecb7d4...,0x8763a8e8f8c968cc05f782f8d90e452a1af789724081...,2023-04-11 15:32:25.193994,2023-04-11 15:32:23.000000
2,6200263,167.172.105.217,mev-boost/v1.5.0 okhttp/4.10.0,2023-04-11 15:32:59.539492+00:00,0xf4bc26ed3cd6ce65fff3e7c249e0d8b3158e1d75c4eb...,0xb0e4825e9b0f07e407c6521d691464a03ec41352edbc...,2023-04-11 15:33:01.098459,2023-04-11 15:32:59.000000
3,6200264,142.132.159.189,mev-boost/v1.5.0,2023-04-11 15:33:11.715400+00:00,0x96a06b84bb0122783f6dc0f671ffa6c93c3b6acd6a68...,0xb2787d518339828927356da0445d0cde6cdcd3255bc1...,2023-04-11 15:33:13.160390,2023-04-11 15:33:11.000000
4,6200265,52.18.21.3,mev-boost/v1.5.0,2023-04-11 15:33:23.510303+00:00,0xdfbbca4c0b929ea48df4a736ddde6052bd4c2546c841...,0x8a446df54f9b40802ee85eb60c528c7544c8fea24292...,2023-04-11 15:33:25.186221,2023-04-11 15:33:23.000000


In [57]:
gp['ua'].values

array(['mev-boost/v1.5.0 Go-http-client/1.1',
       'mev-boost/v1.5.0 Go-http-client/1.1',
       'mev-boost/v1.5.0 okhttp/4.10.0', ..., 'mev-boost/v1.5.0',
       'mev-boost/v1.5.0 Go-http-client/1.1',
       'mev-boost/v1.5.0 Go-http-client/1.1'], dtype=object)

In [58]:
len(gp)

2343

In [59]:
client = Client(project='ultra-sound-relay')

/usr/local/lib/python3.8/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [60]:
getHeaders = pd.DataFrame(columns=['slot', 'ip', 'ua', 'reqTime'])
for v in gp.iterrows():
    time.sleep(1)
    # get successful getPayload calls.
    inAt = v[1]['inserted_at']
    print('querying logs for slot {} inserted at {}'.format(v[1]['slot'], v[1]['inserted_at']), end='\r')
    ghFilter = 'resource.type="http_load_balancer" ' + \
               'httpRequest.status=200 ' + \
               '"https://relay.ultrasound.money/eth/v1/builder/header/{}" '.format(v[1]['slot']) + \
               'jsonPayload.remoteIp="{}" '.format(v[1]['ip']) + \
               'AND timestamp>=' + '\"' + (inAt - pd.Timedelta('12s')).strftime('%Y-%m-%dT%H:%M:%S.%fZ') + '\" ' + \
               'AND timestamp<=' + '\"' + (inAt).strftime('%Y-%m-%dT%H:%M:%S.%fZ') + '\"'
    entries = False
    for e in client.list_entries(filter_=ghFilter):
        entries = True
        out = {}
        out['slot'] = v[1]['slot']
        out['ip'] = e.http_request['remoteIp']
        out['ua'] = e.http_request['userAgent']
        out['reqTime'] = e.timestamp 
        getHeaders = getHeaders.append(out, ignore_index=True)
    if not entries:
        print("\n *** no entries found for slot: {}".format(v[1]['slot']))
#         print(ghFilter)
#         break

querying logs for slot 6200264 inserted at 2023-04-11 15:33:13.160390
 *** no entries found for slot: 6200264
querying logs for slot 6200284 inserted at 2023-04-11 15:37:13.414961
 *** no entries found for slot: 6200284
querying logs for slot 6200288 inserted at 2023-04-11 15:38:03.494606
 *** no entries found for slot: 6200288
querying logs for slot 6200291 inserted at 2023-04-11 15:38:37.115330
 *** no entries found for slot: 6200291
querying logs for slot 6200318 inserted at 2023-04-11 15:44:00.859097
 *** no entries found for slot: 6200318
querying logs for slot 6200352 inserted at 2023-04-11 15:50:50.881609
 *** no entries found for slot: 6200352
querying logs for slot 6200357 inserted at 2023-04-11 15:51:49.308735
 *** no entries found for slot: 6200357
querying logs for slot 6200395 inserted at 2023-04-11 15:59:26.033811
 *** no entries found for slot: 6200395
querying logs for slot 6200409 inserted at 2023-04-11 16:02:13.308597
 *** no entries found for slot: 6200409
querying l

querying logs for slot 6201676 inserted at 2023-04-11 20:15:37.341573
 *** no entries found for slot: 6201676
querying logs for slot 6201696 inserted at 2023-04-11 20:19:39.214621
 *** no entries found for slot: 6201696
querying logs for slot 6201711 inserted at 2023-04-11 20:22:37.613685
 *** no entries found for slot: 6201711
querying logs for slot 6201742 inserted at 2023-04-11 20:28:49.284552
 *** no entries found for slot: 6201742
querying logs for slot 6201746 inserted at 2023-04-11 20:29:37.454346
 *** no entries found for slot: 6201746
querying logs for slot 6201748 inserted at 2023-04-11 20:30:00.998060
 *** no entries found for slot: 6201748
querying logs for slot 6201752 inserted at 2023-04-11 20:30:48.925346
 *** no entries found for slot: 6201752
querying logs for slot 6201776 inserted at 2023-04-11 20:35:37.323813
 *** no entries found for slot: 6201776
querying logs for slot 6201783 inserted at 2023-04-11 20:37:01.291010
 *** no entries found for slot: 6201783
querying l

querying logs for slot 6202583 inserted at 2023-04-11 23:17:01.070205
 *** no entries found for slot: 6202583
querying logs for slot 6202595 inserted at 2023-04-11 23:19:26.260874
 *** no entries found for slot: 6202595
querying logs for slot 6202596 inserted at 2023-04-11 23:19:37.253886
 *** no entries found for slot: 6202596
querying logs for slot 6202630 inserted at 2023-04-11 23:26:25.152333
 *** no entries found for slot: 6202630
querying logs for slot 6202639 inserted at 2023-04-11 23:28:13.515837
 *** no entries found for slot: 6202639
querying logs for slot 6202643 inserted at 2023-04-11 23:29:01.140834
 *** no entries found for slot: 6202643
querying logs for slot 6202645 inserted at 2023-04-11 23:29:24.992502
 *** no entries found for slot: 6202645
querying logs for slot 6202646 inserted at 2023-04-11 23:29:37.654252
 *** no entries found for slot: 6202646
querying logs for slot 6202658 inserted at 2023-04-11 23:32:01.151479
 *** no entries found for slot: 6202658
querying l

querying logs for slot 6203603 inserted at 2023-04-12 02:41:01.525626
 *** no entries found for slot: 6203603
querying logs for slot 6203605 inserted at 2023-04-12 02:41:25.196555
 *** no entries found for slot: 6203605
querying logs for slot 6203608 inserted at 2023-04-12 02:42:01.609392
 *** no entries found for slot: 6203608
querying logs for slot 6203612 inserted at 2023-04-12 02:42:49.966244
 *** no entries found for slot: 6203612
querying logs for slot 6203641 inserted at 2023-04-12 02:48:37.164857
 *** no entries found for slot: 6203641
querying logs for slot 6203661 inserted at 2023-04-12 02:52:37.031137
 *** no entries found for slot: 6203661
querying logs for slot 6203682 inserted at 2023-04-12 02:56:49.166850
 *** no entries found for slot: 6203682
querying logs for slot 6203716 inserted at 2023-04-12 03:03:37.337283
 *** no entries found for slot: 6203716
querying logs for slot 6203718 inserted at 2023-04-12 03:04:01.093903
 *** no entries found for slot: 6203718
querying l

querying logs for slot 6204768 inserted at 2023-04-12 06:34:02.518692
 *** no entries found for slot: 6204768
querying logs for slot 6204777 inserted at 2023-04-12 06:35:49.742590
 *** no entries found for slot: 6204777
querying logs for slot 6204788 inserted at 2023-04-12 06:38:01.185725
 *** no entries found for slot: 6204788
querying logs for slot 6204828 inserted at 2023-04-12 06:46:01.071099
 *** no entries found for slot: 6204828
querying logs for slot 6204864 inserted at 2023-04-12 06:53:14.616010
 *** no entries found for slot: 6204864
querying logs for slot 6204895 inserted at 2023-04-12 06:59:26.041028
 *** no entries found for slot: 6204895
querying logs for slot 6204902 inserted at 2023-04-12 07:00:49.352511
 *** no entries found for slot: 6204902
querying logs for slot 6204926 inserted at 2023-04-12 07:05:36.865782
 *** no entries found for slot: 6204926
querying logs for slot 6204932 inserted at 2023-04-12 07:06:49.104030
 *** no entries found for slot: 6204932
querying l

ResourceExhausted: 429 Quota exceeded for quota metric 'Read requests' and limit 'Read requests per minute' of service 'logging.googleapis.com' for consumer 'project_number:764086051850'. [reason: "RATE_LIMIT_EXCEEDED"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "logging.googleapis.com"
}
metadata {
  key: "quota_metric"
  value: "logging.googleapis.com/read_requests"
}
metadata {
  key: "quota_location"
  value: "global"
}
metadata {
  key: "quota_limit"
  value: "ReadRequestsPerMinutePerProject"
}
metadata {
  key: "quota_limit_value"
  value: "120"
}
metadata {
  key: "consumer"
  value: "projects/764086051850"
}
, links {
  description: "Request a higher quota limit."
  url: "https://cloud.google.com/docs/quota#requesting_higher_quota"
}
]

In [63]:
getHeaders

,slot,ip,ua,reqTime
0,6200251,13.51.48.23,mev-boost/v1.5.0 Go-http-client/1.1,2023-04-11 15:30:35.422391+00:00
1,6200260,34.249.7.41,mev-boost/v1.5.0 Go-http-client/1.1,2023-04-11 15:32:23.365511+00:00
2,6200263,167.172.105.217,mev-boost/v1.5.0 okhttp/4.10.0,2023-04-11 15:32:59.152492+00:00
3,6200265,52.18.21.3,mev-boost/v1.5.0,2023-04-11 15:33:23.212169+00:00
4,6200278,52.191.229.43,mev-boost/v1.5.0 okhttp/4.10.0,2023-04-11 15:35:59.144297+00:00
...,...,...,...,...
1236,6204940,211.197.205.150,mev-boost/v1.5.0 Go-http-client/1.1,2023-04-12 07:08:23.366676+00:00
1237,6204944,20.203.252.23,mev-boost/v1.5.0 okhttp/4.10.0,2023-04-12 07:09:11.113648+00:00
1238,6204945,54.37.30.55,Go-http-client/1.1,2023-04-12 07:09:18.975728+00:00
1239,6204956,95.179.213.218,mev-boost/v1.5.0 okhttp/4.10.0,2023-04-12 07:11:35.100235+00:00


In [65]:
getHeaders.to_csv('ghFull.txt', index=False)

In [62]:
len(set(getHeaders['slot']))

1238

In [14]:
v

slot                                                         6200251
ip                                                       13.51.48.23
ua                               mev-boost/v1.5.0 Go-http-client/1.1
reqTime                             2023-04-11 15:30:35.808494+00:00
block_hash         0xa9919dcc29eabc8e0c967108e306dfc300d2425c7912...
proposer_pubkey    0x8a13257bd79486d55e39b5edba59be7db1539789b71b...
inserted_at                               2023-04-11 15:30:37.358157
slot_boundary                             2023-04-11 15:30:35.000000
Name: 0, dtype: object

In [15]:
v.inserted_at

Timestamp('2023-04-11 15:30:37.358157')

In [24]:
inAt = v.inserted_at
ghFilter = 'resource.type="http_load_balancer" ' + \
           'httpRequest.status=200 ' + \
           '"https://relay.ultrasound.money/eth/v1/builder/header/{}" '.format(v.slot) + \
           'jsonPayload.remoteIp="{}" '.format(v.ip) + \
           'AND timestamp>=' + '\"' + (inAt - pd.Timedelta('12s')).strftime('%Y-%m-%dT%H:%M:%S.%fZ') + '\" ' + \
           'AND timestamp<=' + '\"' + (inAt).strftime('%Y-%m-%dT%H:%M:%S.%fZ') + '\"'

entries = False
for e in client.list_entries(filter_=ghFilter):
    entries = True
    out = {}
    out['slot'] = v.slot
    out['ip'] = e.http_request['remoteIp']
    out['ua'] = e.http_request['userAgent']
    out['reqTime'] = e.timestamp 
#     getPayloads = getPayloads.append(out, ignore_index=True)

In [25]:
out

{'slot': 6200251,
 'ip': '13.51.48.23',
 'ua': 'mev-boost/v1.5.0 Go-http-client/1.1',
 'reqTime': datetime.datetime(2023, 4, 11, 15, 30, 35, 422391, tzinfo=datetime.timezone.utc)}

In [19]:
ghFilter

'resource.type="http_load_balancer" httpRequest.status=200 "https://relay.ultrasound.money/eth/v1/builder/header/6200251" jsonPayload.remoteIp="13.51.48.23" AND timestamp>="2023-04-11T15:30:25.358157Z" AND timestamp<="2023-04-11T15:30:37.358157Z"'

In [ ]:
FILTER = 
'resource.type="http_load_balancer" ' + \
'httpRequest.status=200 ' + \
'"https://relay.ultrasound.money/eth/v1/builder/header/6200251" ' + \
'jsonPayload.remoteIp="68.183.12.3" ' + \
'AND timestamp>=' + '\"' + '2023-04-12T13:05:11.170450Z' '\" ' + \
'AND timestamp<=' + '\"' + '2023-04-12T13:07:11.170450Z' + '\"'